In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
!pip install pycaret

     |████████████████████████████████| 301 kB 18.7 MB/s            
     |████████████████████████████████| 24.2 MB 23.1 MB/s            
     |████████████████████████████████| 636 kB 42.7 MB/s            
     |████████████████████████████████| 25.9 MB 21.3 MB/s            
     |████████████████████████████████| 114 kB 39.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 10.4 MB 22.6 MB/s            
     |████████████████████████████████| 167 kB 39.1 MB/s            
     |████████████████████████████████| 15.6 MB 41.7 MB/s            
     |████████████████████████████████| 184 kB 37.9 MB/s            
     |████████████████████████████████| 1.0 MB 33.0 MB/s            
     |████████████████████████████████| 14.8 MB 19.4 MB/s            
     |████████████████████████████████| 58 kB 7.2 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 79 kB 11.6 MB/s            
  Created wheel fo

In [4]:
from pycaret.regression import *

In [5]:
#import modules
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder,FunctionTransformer
from sklearn.metrics import r2_score
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
%matplotlib inline

In [6]:
KAGGLE=False

In [7]:
if KAGGLE:
    path='../input/jobathonfebruary2022/'
else:
    path=''

In [8]:
train = pd.read_csv(path+'train_0OECtn8.csv')
test = pd.read_csv(path+'test_1zqHu22.csv')
subm = pd.read_csv(path+'sample_submission_JPlpRcN.csv')

In [9]:
print(train.shape)
print(test.shape)

(89197, 10)
(11121, 9)


In [10]:
# def ordinal_dummy_coding(data,cols):
#     data_updated = data.copy()
#     for col in cols:
#         enc_unique = np.sort(data[col].unique())
#         start_enc = enc_unique[0]
#         data_updated = pd.get_dummies(data_updated,columns = [col], prefix=[col])
#         print('after_dummy_update:',data_updated.columns)
#         print('after_dummy_update:',data_updated.head())
#         for enc_val in enc_unique:
#             #remove 1st dummy variable since it will become constant value
#             #as all higher values of 1 will always update it to 1

#             dummy_col = col+'_'+str(enc_val)
#             if enc_val==start_enc:
#                 del data_updated[dummy_col]
#                 continue
                
#             mask = data_updated[dummy_col]==1
#             for prior_enc in range(start_enc+1,enc_val):
#                 prior_dummy_col = col+'_'+str(prior_enc)
#                 data_updated[prior_dummy_col][mask] = 1
            
#     return data_updated    

In [11]:
# print(np.sort(train['followers'].unique()))
# print(np.sort(test['followers'].unique()))

In [12]:
# keys = np.sort(train['followers'].unique())
# values = range(train['followers'].nunique())
# followers_map = dict(zip(keys, values))
# print(followers_map) 

In [13]:
# train['istrain']=1
# test['istrain']=0

# combined = pd.concat([train,test],axis=0)

In [14]:
# combined['followers'] = combined['followers'].replace(followers_map)
# combined= ordinal_dummy_coding(combined,['followers'])

In [15]:
# train= combined[combined['istrain']==1]
# test= combined[combined['istrain']==0]

# del combined,train['istrain'],test['istrain']

In [16]:
print(train.columns)
print(test.columns)

Index(['row_id', 'user_id', 'category_id', 'video_id', 'age', 'gender',
       'profession', 'followers', 'views', 'engagement_score'],
      dtype='object')
Index(['row_id', 'user_id', 'category_id', 'video_id', 'age', 'gender',
       'profession', 'followers', 'views'],
      dtype='object')


In [17]:
def preproc(train,test):
    label_encoder = LabelEncoder()

    train['gender']= label_encoder.fit_transform(train['gender'])
    test['gender']= label_encoder.transform(test['gender'])
    
    #ordinal encoding of profession
    prof_map = {'Other':0,'Student':1,'Working Professional':2}
    train['profession'] = train['profession'].map(prof_map)
    test['profession'] = test['profession'].map(prof_map)
    
    #to have video id without target encoding
    train['video_id_raw']=train['video_id'].copy()
    
def gen_agg(train,test):
    #user_id aggregate features
    user_agg = train[['user_id','video_id','category_id']].groupby('user_id').agg('nunique').reset_index()
    user_agg = user_agg.rename({'video_id':'user_video_count','category_id':'user_category_count'},axis=1)
    train = pd.merge(train, user_agg, how='left', on='user_id')
    test = pd.merge(test, user_agg, how='left', on='user_id')
    
    
    #category_id aggregate features
    cat_agg = train[['category_id','user_id','video_id','age','gender','profession']] \
                        .groupby('category_id') \
                        .agg({'user_id':'nunique','video_id':'nunique', \
                             'age':'mean','gender':'mean','profession':'mean'}) \
                        .reset_index()
    cat_agg = cat_agg.rename({'user_id':'category_user_count',
                              'video_id':'category_video_count',
                              'age':'category_age_mean',
                              'gender':'category_gender_mean',
                              'profession':'category_profession_mean',
                             }, axis=1)
    train = pd.merge(train, cat_agg, how='left', on='category_id')
    test = pd.merge(test, cat_agg, how='left', on='category_id')    
    
    #video_id aggregate features
    video_agg = train[['video_id','age','gender','profession']] \
                        .groupby('video_id') \
                        .agg('mean') \
                        .reset_index() 
    video_agg = video_agg.rename({'age':'video_age_mean',
                              'gender':'video_gender_mean',
                              'profession':'video_profession_mean',
                             },axis=1)
    train = pd.merge(train, video_agg, how='left', on='video_id')
    test = pd.merge(test, video_agg, how='left', on='video_id')
    
    return train,test
        

In [18]:
preproc(train,test)

In [19]:
train,test = gen_agg(train,test)

In [20]:
print(train.columns)
train.head()

Index(['row_id', 'user_id', 'category_id', 'video_id', 'age', 'gender',
       'profession', 'followers', 'views', 'engagement_score', 'video_id_raw',
       'user_video_count', 'user_category_count', 'category_user_count',
       'category_video_count', 'category_age_mean', 'category_gender_mean',
       'category_profession_mean', 'video_age_mean', 'video_gender_mean',
       'video_profession_mean'],
      dtype='object')


,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score,video_id_raw,user_video_count,user_category_count,category_user_count,category_video_count,category_age_mean,category_gender_mean,category_profession_mean,video_age_mean,video_gender_mean,video_profession_mean
0,1,19990,37,128,24,1,1,180,1000,4.33,128,3,2,635,2,24.816008,0.602911,0.922037,24.864811,0.602386,0.912525
1,2,5304,32,132,14,0,1,330,714,1.79,132,5,3,1507,4,24.884218,0.563053,0.862832,25.104735,0.569584,0.892396
2,3,1840,12,24,19,1,1,180,138,4.35,24,3,1,2020,6,24.836205,0.670352,0.883775,25.039640,0.681081,0.935135
3,4,12597,23,112,19,1,1,220,613,3.77,112,3,2,1705,2,25.175964,0.642456,0.897791,25.248317,0.641735,0.893044
4,5,13626,23,112,27,1,2,220,613,3.13,112,4,2,1705,2,25.175964,0.642456,0.897791,25.248317,0.641735,0.893044


In [21]:
targetcol = 'engagement_score'
target = train[targetcol]

In [22]:
exclude_cols =['row_id', 'category_age_mean', 'category_gender_mean',
       'category_profession_mean',  'video_gender_mean',
       'video_profession_mean','video_id_raw',
            'video_age_mean',
        'video_age_ratio', 'video_profession_diff', 
        'video_profession_ratio', 'video_gender_diff', 'video_gender_ratio']
features =[col for col in train.columns if col not in exclude_cols]
print(len(features))
print(features)
features_excl_target = [col for col in features if col!=targetcol]

13
['user_id', 'category_id', 'video_id', 'age', 'gender', 'profession', 'followers', 'views', 'engagement_score', 'user_video_count', 'user_category_count', 'category_user_count', 'category_video_count']


In [23]:
catcols = []
numeric_cols = [col for col in features if col not in catcols+[targetcol]]
print(len(catcols))
print(catcols)
print(len(numeric_cols))
print(numeric_cols)

0
[]
12
['user_id', 'category_id', 'video_id', 'age', 'gender', 'profession', 'followers', 'views', 'user_video_count', 'user_category_count', 'category_user_count', 'category_video_count']


In [24]:
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.woe import WOEEncoder

In [25]:

ct = ColumnTransformer(
     [
         ("targetenc",  TargetEncoder(cols=['user_id','category_id', 'video_id'],
                                                min_samples_leaf=2, smoothing=0.1) , ['user_id','category_id', 'video_id']),
#       ("helmertenc",  HelmertEncoder() , ['followers']),
#     ("targetenc2",  TargetEncoder(cols=['followers'],
#                                                     min_samples_leaf=2, smoothing=0.1) , ['followers']),
#       ("rank",  FunctionTransformer(lambda x: x*10000) , ['followers']),
      ],remainder='passthrough')

# ct2 = ColumnTransformer(
#      [("rank",  FunctionTransformer(lambda x: x.rank()) , ['followers'])
#       ],remainder='passthrough')
preprocessor = ('preproc',ct)

In [26]:
# ct.fit(train,target)
# temp = ct.transform(train)
# temp[0:10,:]

In [27]:
# ce_target=TargetEncoder(cols=['user_id','category_id', 'video_id'],
#                        min_samples_leaf=2, smoothing=0.1)
# ce_target.fit(train,target)

# ret =ce_target.transform(train,target)
# ret.head()

In [28]:
s = setup(train[features], session_id=100,
          silent=True,
          numeric_features=numeric_cols,
          categorical_features=catcols,
          target = targetcol,
          custom_pipeline=preprocessor,
          fold=10,fold_shuffle=False,
          use_gpu=True,
          )

,Description,Value
0,session_id,100
1,Target,engagement_score
2,Original Data,"(89197, 13)"
3,Missing Values,False
4,Numeric Features,12
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(62437, 12)"


In [29]:
X= get_config('X')
print(len(X.columns))
print(list(X.columns))
X.head()

12
['user_id', 'category_id', 'video_id', 'age', 'gender', 'profession', 'followers', 'views', 'user_video_count', 'user_category_count', 'category_user_count', 'category_video_count']


,user_id,category_id,video_id,age,gender,profession,followers,views,user_video_count,user_category_count,category_user_count,category_video_count
0,19990.0,37.0,128.0,24.0,1.0,1.0,180.0,1000.0,3.0,2.0,635.0,2.0
1,5304.0,32.0,132.0,14.0,0.0,1.0,330.0,714.0,5.0,3.0,1507.0,4.0
2,1840.0,12.0,24.0,19.0,1.0,1.0,180.0,138.0,3.0,1.0,2020.0,6.0
3,12597.0,23.0,112.0,19.0,1.0,1.0,220.0,613.0,3.0,2.0,1705.0,2.0
4,13626.0,23.0,112.0,27.0,1.0,2.0,220.0,613.0,4.0,2.0,1705.0,2.0


In [48]:
#  params = {
#      'bagging_fraction': 0.8,
#     'bagging_freq': 15,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#      'max_depth': 12,
#     'num_leaves': 30,
#     'reg_alpha': 0.3,  
#     'reg_lambda': 0.7,
#      'n_estimators': 200,
#  }

In [47]:
# %%time
# lgbm = create_model('lightgbm',**params)
# lgbm

In [46]:
# # tune_params = {"max_depth": np.random.randint(1, (len(data.columns)*.85),20),
# #           "max_features": np.random.randint(1, len(train.columns),20),
# #           "min_samples_leaf": [20,50,100,500],
# #           "criterion": ["gini", "entropy"]
# #           }

# tune_params = {
#     'bagging_fraction': np.random.uniform( 0.6, 1, 25),
#     'num_leaves': np.random.randint( 20, 150, 5),
#     'subsample_for_bin': np.random.randint( 40000, 160000, 10000),
#     'min_child_samples': np.random.randint(30, 200, 5),
#     'reg_alpha': np.random.uniform( 0.01, 0.99),
#     'reg_lambda': np.random.uniform( 0.01, 0.99),
#     'feature_fraction': np.random.uniform(0.6, 1.0),
# }

# # bagging_freq(subsample_freq), bagging_fraction (subsample), feature_fraction (colsample_bytree)

In [45]:
# %%time
# tuned_lgbm = tune_model(lgbm,optimize='r2',
#                         search_library='scikit-optimize',
#                         n_iter=25)
# tuned_lgbm

In [44]:
# save_model(tuned_lgbm, 'tuned_lgbm_model')

In [42]:
# lgbm_old = lgbm
# lgbm= tuned_lgbm

In [43]:
# lgbm

In [41]:
# plot_model(lgbm,'feature_all')

xgboost model

In [32]:
%%time
xgbm=create_model('xgboost',base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.020277635007304968,
             max_delta_step=0, max_depth=11, min_child_weight=4,
             monotone_constraints='()', n_estimators=300, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=100, reg_alpha=1e-10,
             reg_lambda=1e-10, scale_pos_weight=46.751325738834,
             subsample=0.9840433785610212, tree_method='gpu_hist',
             validate_parameters=1, verbosity=0)
xgbm

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.4891,0.4481,0.6694,0.4253,0.1891,0.2368
1,0.4926,0.4372,0.6612,0.4073,0.1793,0.2366
2,0.5006,0.4610,0.6790,0.4021,0.1880,0.2775
3,0.4927,0.4422,0.6650,0.4012,0.1821,0.2208
4,0.4923,0.4426,0.6653,0.4162,0.1832,0.1920
5,0.4978,0.4507,0.6713,0.3956,0.1819,0.2210
6,0.4858,0.4238,0.6510,0.4196,0.1746,0.1866
7,0.4942,0.4428,0.6654,0.4116,0.1823,0.2315
8,0.4847,0.4243,0.6514,0.4138,0.1760,0.2339
9,0.4925,0.4420,0.6648,0.4152,0.1844,0.2531


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.020277635007304968,
             max_delta_step=0, max_depth=11, min_child_weight=4, missing=nan,
             monotone_constraints='()', n_estimators=300, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=100, reg_alpha=1e-10,
             reg_lambda=1e-10, scale_pos_weight=46.751325738834,
             subsample=0.9840433785610212, tree_method='gpu_hist',
             validate_parameters=1, verbosity=0)

In [33]:
save_model(xgbm, 'tuned_xgb_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['user_id',
                                                           'category_id',
                                                           'video_id', 'age',
                                                           'gender',
                                                           'profession',
                                                           'followers', 'views',
                                                           'user_video_count',
                                                           'user_category_count',
                                                           'category_user_count',
                                      

In [40]:
# extratrees=create_model('et')
# extratrees

In [39]:
# save_model(extratrees, 'et_model')

In [36]:
# catboost=create_model('catboost')
# catboost

In [35]:
# save_model(catboost, 'catboost_model')

In [37]:
# rf=create_model('rf')
# rf

In [38]:
# save_model(rf, 'randomforest_model')

Ensemble models

In [49]:
# stacked_model = stack_models([tuned_lgbm,xgbm,catboost,rf],
# #                               meta_model=
#                             ),
# stacked_model

In [50]:
# save_model(stacked_model, 'stacked_model')

Predictions

In [34]:
%%time
#test predictions
# final_model = finalize_model(stacked_model[0])
final_model = finalize_model(xgbm)
test_preds = predict_model(final_model, data=test)
print(test_preds.head())

   row_id  user_id  category_id  video_id  age  gender  profession  followers  \
0   89198     7986           12        42   14       1           1        180   
1   89199    11278           34       115   14       1           1        230   
2   89200    17245            8       110   44       0           2        280   
3   89201     9851           16       137   18       1           1        270   
4   89202    16008           34        96   47       0           0        230   

   views  user_video_count  user_category_count  category_user_count  \
0    138                 2                    1                 2020   
1    840                 5                    3                 1679   
2    628                 5                    2                 3043   
3    462                 3                    2                 1670   
4    840                 3                    2                 1679   

   category_video_count  category_age_mean  category_gender_mean  \
0           

In [51]:
final_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.020277635007304968,
             max_delta_step=0, max_depth=11, min_child_weight=4, missing=nan,
             monotone_constraints='()', n_estimators=300, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=100, reg_alpha=1e-10,
             reg_lambda=1e-10, scale_pos_weight=46.751325738834,
             subsample=0.9840433785610212, tree_method='gpu_hist',
             validate_parameters=1, verbosity=0)

In [52]:
# %%time
# #train predictions
# train_preds = predict_model(final_model, data=train)
# print(train_preds.head())
# print('full train score:',r2_score(target,train_preds['Label']))

In [53]:
submission = test_preds[['row_id','Label']]
submission=submission.rename(columns={'Label':'engagement_score'})
submission[['row_id','engagement_score']].to_csv('submission.csv',index=False)

In [54]:
test_sub = pd.read_csv('submission.csv')

In [55]:
print(test_sub.shape)
test_sub.head()

(11121, 2)


,row_id,engagement_score
0,89198,4.200339
1,89199,4.019956
2,89200,2.424595
3,89201,2.863360
4,89202,1.939223
